In [ ]:
import scripts.cmcomp
%load_ext autoreload
%autoreload 2
%matplotlib inline
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs
import subprocess
import seaborn as sns
from umap import UMAP
from yoda import draw
import ubergauss.optimization as uo 
import pandas as pd
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount
import ubergauss.tools as ut
from yoda import graphs as ygraphs

In [ ]:
# GENERATE CM FILES
def dumpcm(family_id1):
    # Extract alignments from Stockholm file based on family IDs
    align1_path = f'{family_id1}.fasta'
    with open('/home/ikea/Rfam.seed.utf8', 'r') as stockholm_file:
        found1 = False
        with open(align1_path, 'w') as align1_file:
            align1_file.write('# STOCKHOLM 1.0\n\n')
            for line in stockholm_file:
                if line.startswith(f'#=GF AC') and family_id1 in line:
                    found1 = True
                elif line.startswith('//'):
                    if found1:
                        align1_file.write('//')
                        break
                    found1 = False
                elif found1:
                    align1_file.write(line)
    # Generate cm files from fasta alignments
    cm1_path = align1_path.replace('.fasta', '.cm')
    infernal_cmd1 = f'cmbuild -F --informat stockholm {cm1_path} {align1_path}'
    subprocess.run(infernal_cmd1, shell=True, check=True)
    return True
    
# RUN CMCOMPARE 
def compare_cm(_,cm1=None, cm2=None):
    cm1 = names[cm1]
    cm2 = names[cm2]
    compare_cmd = f'/home/ikea/Downloads/hsCMCompare-archlinux-x64 {cm1}.cm {cm2}.cm'
    result = subprocess.run(compare_cmd, shell=True, check=True, capture_output=True, text=True)
    #print(result.stdout)
    score = result.stdout.split()[3]  
    score2 = result.stdout.split()[2]  
    #score = float(score_line.split()[2]) 
    return {'score':float(score),'score2': float(score2)}

def run_cmcompare_pairwise():
    num_cm = len(names) # or just use 10 for debugging :) 
    return uo.gridsearch(compare_cm,
                    param_dict = {'cm1':lmz.Range(num_cm), 'cm2':lmz.Range(num_cm)},
                    data = [False], 
                    taskfilter = lambda x: x['cm1'] <= x['cm2']) 

def load_complete_run():
    # delete this
    res = pd.read_csv('CMCOMPARE_RESULTS')
    res.loc[len(res)] = [0,0,0,0,0]# should give us cm2 = 0 
    res.loc[len(res)] = [0,393,393,0,0]# should give us cm1 = 393
    return res
    
def load_cmcompare_better():
    return pd.read_csv('cmcompare_full_run')


In [ ]:
#IMPORTANTRESULT = run_cmcompare_pairwise()
#IMPORTANTRESULT.to_csv('cmcompare_full_run')

In [ ]:
a,l = load_rfam(full= False, add_cov= False)
names = [aa.gf["AC"][3:] for aa in a]
# [dumpcm(name) for name in names] 
res = pd.read_csv('cmcompare_full_run_2024_06_27')
df_filled = df.fillna(500) 


# res = load_complete_run()
if False:
    df = res_nu
    print(df[df.isna().any(axis=1)])
if False:
    res = load_cmcompare_better()
    def fix_load_cmcompare_better(a,l):
        return lmz.Transpose([ (aa,ll) for aa,ll in zip(a,l) if 'ID snR56' != aa.gf['ID'] ])
    a,l = fix_load_cmcompare_better(a,l)
    names = [aa.gf["AC"][3:] for aa in a]

res['score1'] = res[['score', 'score2']].min(axis=1)
res['score3'] = res[['score', 'score2']].max(axis=1)

In [ ]:
# EVAL 
import yoda.ml.simpleMl as sml

def to_numpy(df,l):
    
    p = df.pivot(index='cm1', columns='cm2', values='score1')
    
    for i in range(p.values.shape[0]):
        for j in range(i+1,p.values.shape[0]):
            p.iloc[j,i] = p.iloc[i,j]
            
    np.fill_diagonal(p.values,0)

    cmap = plt.cm.get_cmap('viridis', max(l) - min(l) + 1)
    colors = [cmap(i - min(l)) for i in l]
    
    sns.clustermap(p, row_colors= colors)
    plt.show()
    p= p.values
    p+=p.min()
    return p

def embed(X,y):
    emb = UMAP().fit_transform(X)
    draw.scatter(emb,y)

def score(X,l,k):
    # X is the similarity matrix
    # np.fill_diagonal(X,X.max()+0.05)
    X = -X
    X-=X.min()
    np.fill_diagonal(X,0)
    print(X[:10,:10])
    print('AVG PREC:', sml.average_precision(X,l))
    data  = [sml.clan_in_x(X,l,n) for n in range(k)]
    plt.plot(data)
    return data,X

X = to_numpy(res,l)
embed(X,l)
scores_cmcomp,DIST_CM = score(X,l,50)
avg_cmcomp = sml.average_precision_srt(X,l)

In [ ]:
def atograph(ali):
    graph = ali2graph.writecons(ali)
    graph = ali2graph.dillute( graph, dilute1 = .7, dilute2 = .4 , fix_edges =True)
    # graph = ali2graph.conscut( graph,  consThresh= .95, replacelabel = False )
    graphs = ali2graph.set_weight( graph,  consThresh = .97 )
    # graphs = ali2graph.nearstem( graph, boost_range = 4, boost_thresh = .5, boost_weight = 1)
    graphs = ali2graph.donest( graph)
    return graph



graphs = ut.xxmap(atograph, a)
matrix = ygraphs.vectorize_graphs(graphs, normalization=True,
                      min_r = 2,min_d=1,d=3,
                      inner_normalization=True)



In [ ]:

import pandas as pd
from lmz import Range
from sklearn.metrics.pairwise import euclidean_distances


def mkdf(mat): 
    sim = mat @ mat.T
    # sim = -euclidean_distances(mat)
    data = [{'cm1':a, 'cm2':b , 'score1': sim[a,b]} for a in Range(l) for b in Range(l)]
    return pd.DataFrame(data)
    
df = mkdf(matrix)

In [ ]:
X = to_numpy(df,l)
embed(X,l)
myscores, DIST_US = score(X,l,50)
avg_us = sml.average_precision_srt(X,l)

from sklearn.preprocessing import normalize
X = normalize(X, axis=0)
myscores_, DIST_US2 = score(X,l,50)

In [ ]:
niigpudata =  [0.0, 0.4847715736040609, 0.5279187817258884, 0.565989847715736, 0.5862944162436549, 0.6015228426395939, 0.6294416243654822, 0.6395939086294417, 0.649746192893401, 0.6649746192893401, 0.6776649746192893, 0.6852791878172588, 0.6928934010152284, 0.700507614213198, 0.7055837563451777, 0.7131979695431472, 0.7208121827411168, 0.7258883248730964, 0.733502538071066, 0.7360406091370558, 0.7436548223350253, 0.7538071065989848, 0.7563451776649747, 0.7614213197969543, 0.7639593908629442, 0.7639593908629442, 0.7741116751269036, 0.7791878172588832, 0.7817258883248731, 0.7918781725888325, 0.7918781725888325, 0.7918781725888325, 0.7944162436548223, 0.799492385786802, 0.8020304568527918, 0.8020304568527918, 0.8071065989847716, 0.8071065989847716, 0.8071065989847716, 0.8121827411167513, 0.8121827411167513, 0.817258883248731, 0.8223350253807107, 0.8248730964467005, 0.8274111675126904, 0.8274111675126904, 0.8274111675126904, 0.8274111675126904, 0.8350253807106599, 0.8350253807106599]
plt.plot(niigpudata[1:])
plt.plot(scores_cmcomp[1:])
plt.plot(myscores[1:])
plt.plot(myscores_[1:])
# our line should look like the other one

In [ ]:
plt.plot(avg_cmcomp)
plt.plot(avg_us)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score
# try the new experimental kernel :) 
# https://github.com/Xfcosta/CoCoGraPE_notebooks/blob/main/data_processor/supervised/graph_kernels_behaviour.ipynb

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt

def get_pairwise_distances(dist_matrix, labels):
    pairwise_distances = []
    pairwise_labels = []
    n = dist_matrix.shape[0]

    for i in range(n):
        for j in range(i+1, n):
            pairwise_distances.append(dist_matrix[i][j])
            pairwise_labels.append(labels[i] == labels[j]) 

    return np.array(pairwise_distances), np.array(pairwise_labels)


def prec_recall_plot(distances, labels):
    distances, true_labels = get_pairwise_distances(distances, labels)
    precision, recall, thresholds = precision_recall_curve(true_labels, -distances) 
    print(f'{auc(recall, precision)=}')
    # Plot Precision-Recall curve
    plt.figure()
    plt.plot(recall, precision, marker='.')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

prec_recall_plot(DIST_US,l)
prec_recall_plot(DIST_US2,l)
prec_recall_plot(DIST_CM,l)

In [ ]:
import inferencemodel as inf
SIM_INF = scripts.cmcomp.readCmscanAndMakeTable((a, l), path ='inftools/infernal.tbl')
scores_INF,DIST_INF = score(SIM_INF,l,50)

indices = np.where(DIST_INF == 10000)
noise = np.random.normal(0, 1, size=len(indices[0]))
DIST_INF[indices] += noise


prec_recall_plot(DIST_INF,l)
plt.matshow(DIST_INF)
plt.show()

In [ ]:
plt.matshow(SIM_INF[:100,:100])